In [ ]:
from tensorflow import keras

## Transfer Learning 을 통한 가위-바위-보 분류 데이터 셋 분류 성능 개선

### Step 1. Input tensor 와 Target tensor 준비(훈련데이터)

(1) 가위-바위-보 데이터셋 다운로드

In [ ]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip'

path_to_zip = keras.utils.get_file('rps.zip',
                                   origin=url,
                                   extract=True,cache_dir='/content')

200687616/200682221 [==============================] - 1s 0us/step


In [ ]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip'

path_to_zip = keras.utils.get_file('rps_test.zip',
                                   origin=url,
                                   extract=True, cache_dir='/content')

29523968/29516758 [==============================] - 0s 0us/step


(2) ImageDataGenerator를 이용해 이미지 파일을 load 하기 위한 경로 지정

In [ ]:
train_dir = '/content/datasets/rps'
test_dir = '/content/datasets/rps-test-set'

(3) ImageDataGenerator 객체 생성  
* 객체 생성 시 rescale 인자를 이용하여 텐서 내 원소의 범위를 [0 ~ 255] => [0 ~ 1] 로 ReScaling 진행

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255.)
test_datagen = ImageDataGenerator(rescale=1./255.)

* .flow_from_directory() 메서드를 이용하여 학습데이터와 검증데이터를 위한 DirectoryIterator 객체 생성

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=20,
    shuffle=True,
    class_mode='categorical',
    subset='training',
    seed=7)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224),
    batch_size=20,
    shuffle=True,
    class_mode='categorical',
    subset='validation', seed=7)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224,224),
    batch_size=20,
    class_mode='categorical'
)

Found 2520 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


### Step 2. VGG16을 Backbone 으로 하는 모델 디자인 및 학습 정보 설정

(1) Pre-trained 된 VGG16 모델 객체 생성
  * imagenet 데이터를 이용해 학습된 모델 객체 생성
  * classification layer 제외

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
conv_base = VGG16(include_top=False, weights = 'imagenet', input_shape=(224,224,3))

58892288/58889256 [==============================] - 1s 0us/step


(2) VGG16 Backbone 모델에 classification layer 추가

In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

(3) VGG16 Backbone 모델의 가중치 동결(학습대상 가중치에서 제외)

* VGG16 Backbone 모델의 가중치 동결 및 동결 후 학습대상 파라미터 개수 출력

In [ ]:
model = keras.Sequential()

(4) 학습을 위한 설정 정보 지정

In [ ]:
model.add(conv_base)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 21,138,243
Trainable params: 21,138,243
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#block5_conv3 layer만 제외하고 동결
for layer in conv_base.layers:
  if layer.name=='block5_conv3':
    continue
  layer.trainable=False

In [ ]:
len(model.trainable_weights)

6

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               6422784   
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 21,138,243
Trainable params: 8,783,363
Non-trainable params: 12,354,880
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['accuracy'])

### Step 3. 모델에 데이터 generator 연결 후 학습 
  * model.fit() 이용하여 데이터 연결 및 학습시키기
  * 학습 과정은 history 변수에 저장

In [ ]:
history = model.fit(train_generator,
          steps_per_epoch=len(train_generator), 
          epochs=10,
          validation_data = validation_generator,
          validation_steps = len(validation_generator))

Epoch 1/10
126/126 [==============================] - 14s 114ms/step - loss: 5.4306e-08 - accuracy: 1.0000
Epoch 2/10
126/126 [==============================] - 15s 116ms/step - loss: 4.7731e-08 - accuracy: 1.0000
Epoch 3/10
126/126 [==============================] - 14s 115ms/step - loss: 4.2717e-08 - accuracy: 1.0000
Epoch 4/10
126/126 [==============================] - 14s 113ms/step - loss: 3.6662e-08 - accuracy: 1.0000
Epoch 5/10
126/126 [==============================] - 14s 113ms/step - loss: 3.3587e-08 - accuracy: 1.0000
Epoch 6/10
126/126 [==============================] - 14s 113ms/step - loss: 3.0323e-08 - accuracy: 1.0000
Epoch 7/10
126/126 [==============================] - 14s 113ms/step - loss: 2.7248e-08 - accuracy: 1.0000
Epoch 8/10
126/126 [==============================] - 14s 114ms/step - loss: 2.5450e-08 - accuracy: 1.0000
Epoch 9/10
126/126 [==============================] - 14s 114ms/step - loss: 2.2801e-08 - accuracy: 1.0000
Epoch 10/10
126/126 [================

### Step 4. 테스트 데이터 셋을 통한 모델의 성능 평가

In [ ]:
loss, accuracy = model.evaluate(test_generator)

19/19 [==============================] - 5s 264ms/step - loss: 0.1644 - accuracy: 0.9382
